In [2]:
!pip install stop_words

In [3]:
!pip install pymorphy2

In [4]:
!pip install navec

In [5]:
import numpy as np
import pandas as pd
pd.set_option('max_colwidth', 100)
import re
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from navec import Navec
from gensim.models import KeyedVectors

import nltk
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
from google.colab import files
uploaded = files.upload()

Saving отзывы за лето.xls to отзывы за лето.xls


In [6]:
df = pd.read_excel('отзывы за лето.xls')
df.head(10)

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотят слишком большой доступ к персональным данным в т...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше никуда. Ранее больше года пользовался нормально.,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14
5,5,Всё удобно норм 👍👍👍,2017-08-14
6,5,Очень удобное приложение.,2017-08-14
7,5,Все устраивает,2017-08-14
8,5,"У меня работает все четко. В отличии от банкоматов, которые вечно зависают и тупят",2017-08-14
9,5,Очень все хорошо👍,2017-08-14


In [7]:
max_words = 200
max_len = 140
num_classes = 1

epochs = 20
batch_size = 512
print_batch_n = 100

In [8]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)
    
df['Content'] = df['Content'].apply(preprocess_text)

In [9]:
df.head(10)

,Rating,Content,Date
0,5,it just works,2017-08-14
1,4,целое удобноной приложениеиз минус хотеть большой доступ персональный данные телефонеприходиться...,2017-08-14
2,5,отлично,2017-08-14
3,5,зависать 1 работа антивирус ранее пользоваться нормально,2017-08-14
4,5,удобно работать быстро,2017-08-14
5,5,удобно норма 👍👍👍,2017-08-14
6,5,удобный приложение,2017-08-14
7,5,устраивать,2017-08-14
8,5,работать чётко отличие банкомат вечно зависать тупить,2017-08-14
9,5,хорошо👍,2017-08-14


In [10]:
df['Rating'].nunique()

5

In [11]:
df_train, df_test = train_test_split(df, test_size=0.2)
df_test, df_val = train_test_split(df_test, test_size=0.2)
print(f'Train: {df_train.shape}, Test: {df_test.shape}, Val: {df_val.shape}')

Train: (16527, 3), Test: (3305, 3), Val: (827, 3)


In [12]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

In [13]:
def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

In [14]:
df_train['Content'] = df_train['Content'].apply(preprocess_text)
df_val['Content'] = df_val['Content'].apply(preprocess_text)
df_test['Content'] = df_test['Content'].apply(preprocess_text)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [15]:
train_corpus = " ".join(df_train["Content"])
train_corpus = train_corpus.lower()

In [16]:
tokens = word_tokenize(train_corpus)

In [17]:
tokens_filtered = [word for word in tokens if word.isalnum()]

In [18]:
dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]

In [19]:
tokens_filtered_top[:10]

['приложение',
 'удобно',
 'работать',
 'удобный',
 'отлично',
 'нравиться',
 'хороший',
 'отличный',
 'телефон',
 'супер']

In [20]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [21]:
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [22]:
x_train = np.asarray([text_to_sequence(text, max_len) for text in df_train["Content"]], dtype=np.int32)
x_test = np.asarray([text_to_sequence(text, max_len) for text in df_test["Content"]], dtype=np.int32)
x_val = np.asarray([text_to_sequence(text, max_len) for text in df_val["Content"]], dtype=np.int32)

In [23]:
df_train["Rating"] = df_train["Rating"] - 1
df_val["Rating"] = df_val["Rating"] - 1
df_test["Rating"] = df_test["Rating"] - 1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [24]:
x_train[1]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
        21,   9,  22,  52,  67, 146,  78,  20, 157, 124], dtype=int32)

In [25]:
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Bidirectional, LSTM, SpatialDropout1D, concatenate
from keras.layers import Dropout
from keras.layers import Activation
from keras.layers import Input
from keras.layers import Embedding
from keras.layers import Conv1D, Flatten, MaxPooling1D
from keras.layers import GlobalMaxPool1D, GlobalAveragePooling1D, GlobalMaxPooling1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
from keras.objectives import categorical_crossentropy
from keras.callbacks import EarlyStopping 
import tensorflow as tf

In [46]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(Conv1D(128, 3))
model.add(Bidirectional(LSTM(units=32, return_sequences=True)))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [35]:
num_classes = 5
y_train = tf.keras.utils.to_categorical(df_train["Rating"], num_classes=num_classes)
y_val = tf.keras.utils.to_categorical(df_val["Rating"], num_classes=num_classes)

In [36]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [48]:
tensorboard = TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping = EarlyStopping(monitor='val_loss')  


history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

RuntimeError: ignored

In [44]:
score = model.evaluate(x_val, y_val, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

ValueError: ignored

In [54]:
ru_w2v = KeyedVectors.load_word2vec_format('https://rusvectores.org/static/models/news_upos_cbow_300_2_2017.bin.gz', binary=True)

In [55]:
def code_w2v_txt(txt, max_len = 100):
    sent_w2v = []
    zero_point = np.zeros(300)
    txt = txt.split()
   
    for i in range(max_len):
        try:
            word = ru_w2v[txt[i]]
        except:
            word = zero_point
        sent_w2v.append(word)
    return np.array(sent_w2v)

max_len = 50
df['w2v'] = df['Content'].apply(code_w2v_txt, max_len = max_len)
del(ru_w2v)

In [56]:
X = list(df['w2v'].values)
X = np.array(X)
y = df['Rating'].values

In [57]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [60]:
le = LabelEncoder()
train_enc_labels = le.fit_transform(y_train) 
test_enc_labels = le.transform(y_test)
le.classes_

array([1, 2, 3, 4, 5])

In [61]:
num_classes = len(df.Rating.unique())

inputs = Input(shape=(X_train.shape[1], X_train.shape[2]))
dropout_embeds = SpatialDropout1D(0.05)(inputs)
x = Bidirectional(LSTM(units=7, return_sequences=True))(dropout_embeds)
pooled_avg_sequences = GlobalAveragePooling1D()(x)
pooled_max_sequences = GlobalMaxPooling1D()(x)
concated = concatenate([pooled_avg_sequences, pooled_max_sequences])
dense_intermediate = Dense(64, activation='elu')(concated)
x = Dense(num_classes, activation='sigmoid')(dense_intermediate)

model = Model(inputs=inputs, outputs=x)
model.summary()
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              optimizer='adam',
              metrics=['accuracy'])

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 50, 300)]    0                                            
__________________________________________________________________________________________________
spatial_dropout1d (SpatialDropo (None, 50, 300)      0           input_1[0][0]                    
__________________________________________________________________________________________________
bidirectional_2 (Bidirectional) (None, 50, 14)       17248       spatial_dropout1d[0][0]          
__________________________________________________________________________________________________
global_average_pooling1d (Globa (None, 14)           0           bidirectional_2[0][0]            
______________________________________________________________________________________________

In [62]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(X_train, train_enc_labels,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
26/26 [==============================] - 17s 111ms/step - loss: 1.5529 - accuracy: 0.6224 - val_loss: 1.2974 - val_accuracy: 0.7049


In [63]:
score = model.evaluate(X_test, test_enc_labels)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

194/194 [==============================] - 1s 5ms/step - loss: 1.2966 - accuracy: 0.7086


Test score: 1.2966086864471436
Test accuracy: 0.7086156606674194


In [ ]:
inputs = Input(shape=(X_train.shape[1], X_train.shape[2]))
l_cov1= Conv1D(128, 5, activation='relu')(inputs)
l_pool1 = MaxPooling1D(2)(l_cov1)
l_cov2 =Conv1D(128, 5, activation='relu')(l_pool1)
l_pool2 = MaxPooling1D(2)(l_cov2)
l_cov3 = Conv1D(128, 5, activation='relu')(l_pool2)
l_pool3 = MaxPooling1D(5)(l_cov3)  
l_flat = Flatten()(l_pool3)
l_dense = Dense(128, activation='relu')(l_flat)
preds = Dense(64, activation='softmax')(l_dense)
preds = Dense(num_classes, activation='softmax')(l_dense)

model = Model(inputs=inputs, outputs=preds)

model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(X_train, train_enc_labels,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
 1/26 [>.............................] - ETA: 11:37 - loss: 1.6094 - accuracy: 0.0938

In [ ]:
score = model.evaluate(X_test, test_enc_labels)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])